In [7]:
import pandas as pd
from numpy import linalg as LA
from sklearn import decomposition, preprocessing

Data = pd.read_csv('HW1_census-tract_data.csv')
Data.loc[0,:]
Data

,Tract,Total poupulation(thousands),Median school years,Total employment(thousands),Health services employment(hundreds),"Median value home($10,000s)"
0,1,5.935,14.2,2.265,2.27,2.91
1,2,1.523,13.1,0.597,0.75,2.62
2,3,2.599,12.7,1.237,1.11,1.72
3,4,4.009,15.2,1.649,0.81,3.02
4,5,4.687,14.7,2.312,2.50,2.22
5,6,8.044,15.6,3.641,4.51,2.36
6,7,2.766,13.3,1.244,1.03,1.97
7,8,6.538,17.0,2.618,2.39,1.85
8,9,6.451,12.9,3.147,5.52,2.01
9,10,3.314,12.2,1.606,2.18,1.82


In [8]:
import numpy as np
np.set_printoptions(precision=3)
# Change the unit of 'Medina value home" from ten thousands to hundred.
# Data.loc[:,'Median value home($10,000s)'] *= 100 # (10,000/100)

# Convert Dataframe to numpy array, drop the irrelevant column "tract".
Drop_Tract = Data.drop(columns=['Tract'])
# Convert Dataframe to numpy
Column6_Multiplied_by_100 = Drop_Tract.to_numpy()
# Transpose the data for covariance matrix computation.
Column6_Multiplied_by_100 = np.transpose(Column6_Multiplied_by_100)

# Check the shape of data after converting.
#np.array(Column6_Multiplied_by_100).shape

MSY = Data["Median school years"]
HSE = Data["Health services employment(hundreds)"]

In [9]:
Covariance_Matrix_S = np.cov(Column6_Multiplied_by_100)
print(f'shape of the Covariance Matrix: {np.array(Covariance_Matrix_S).shape}')
print(f'Covariance Matrix:')
Covariance_Matrix_S

shape of the Covariance Matrix: (5, 5)
Covariance Matrix:


array([[ 4.308,  1.684,  1.803,  2.155, -0.253],
       [ 1.684,  1.767,  0.588,  0.178,  0.176],
       [ 1.803,  0.588,  0.801,  1.065, -0.158],
       [ 2.155,  0.178,  1.065,  1.969, -0.357],
       [-0.253,  0.176, -0.158, -0.357,  0.504]])

##### (b)-1, Obtain eigenvalue-eigenvector pairs

In [10]:
eigenvalues, eigenvectors = LA.eig(Covariance_Matrix_S)

# sort the eigenvalues in ascending order.
index = np.flip(np.argsort(eigenvalues))
eigenvalues = eigenvalues[index]
eigenvectors = eigenvectors[:,index]

print(f'eigenvalues:\n{eigenvalues}')
print(f'eigenvectors:\n{eigenvectors}')

# a = eigenvectors[0,:]
# b = eigenvectors[1,:]
# print(a)
# print(b)
# print(np.inner(a,b))

eigenvalues:
[6.931 1.785 0.39  0.23  0.014]
eigenvectors:
[[-0.781 -0.071  0.004 -0.542  0.302]
 [-0.306 -0.764 -0.162  0.545  0.009]
 [-0.334  0.083  0.015 -0.051 -0.937]
 [-0.426  0.579  0.22   0.636  0.172]
 [ 0.054 -0.262  0.962 -0.051 -0.025]]


---
##### (b)-2, Obtain the first two sample principal components for the covariance matrix.
* reference: https://stackoverflow.com/questions/43355044/cumulative-explained-variance-for-pca-in-python

In [11]:
# Normalize the covariance matrix
# pca_data = Covariance_Matrix_S
# pca_data = preprocessing.normalize(Column6_Multiplied_by_100)
Column6_Multiplied_by_100 = np.transpose(Column6_Multiplied_by_100)
pca_data = Column6_Multiplied_by_100
#print(pca_data)

# Principal_Components = pca.transform(pca_data)

pca = decomposition.PCA(n_components=5, svd_solver="full")
print(pca)
pca.fit(pca_data)
print(f'Components:\n  {pca.components_}')
print(f'(explained) Variance:\n  {pca.explained_variance_}')
print(f'Ratio of (explained) Variance:\n  {pca.explained_variance_ratio_}')

cum_explained_var = []
for i in range(0, len(pca.explained_variance_ratio_)):
    if i == 0:
        cum_explained_var.append(pca.explained_variance_ratio_[i])
    else:
        cum_explained_var.append(pca.explained_variance_ratio_[i]
                                 + cum_explained_var[i-1])

print(f'Cumulative percentage of total variance:\n  {cum_explained_var}')

PCA(n_components=5, svd_solver='full')
Components:
  [[ 0.781  0.306  0.334  0.426 -0.054]
 [-0.071 -0.764  0.083  0.579 -0.262]
 [ 0.004 -0.162  0.015  0.22   0.962]
 [ 0.542 -0.545  0.051 -0.636  0.051]
 [-0.302 -0.009  0.937 -0.172  0.025]]
(explained) Variance:
  [6.931 1.785 0.39  0.23  0.014]
Ratio of (explained) Variance:
  [0.741 0.191 0.042 0.025 0.002]
Cumulative percentage of total variance:
  [0.7413268330424228, 0.9322605150253773, 0.9739363013351573, 0.9984860254881521, 0.9999999999999999]


In [12]:
# Calculate the correlation coefficients
Correlation_coefficients_matrix = np.corrcoef(Column6_Multiplied_by_100)
print(Correlation_coefficients_matrix)

MSY_var = np.var(MSY)   #1.6412
HSE_var = np.var(HSE)   #1.8288

# print(HSE_var)

[[1.    0.956 0.982 0.987 0.992 0.978 0.983 0.996 0.942 0.985 0.98  0.926
  0.974 0.998]
 [0.956 1.    0.99  0.987 0.972 0.895 0.992 0.944 0.871 0.98  0.97  0.993
  0.996 0.947]
 [0.982 0.99  1.    0.994 0.995 0.946 1.    0.979 0.922 0.997 0.992 0.968
  0.998 0.975]
 [0.987 0.987 0.994 1.    0.988 0.938 0.996 0.977 0.897 0.987 0.978 0.971
  0.992 0.985]
 [0.992 0.972 0.995 0.988 1.    0.974 0.993 0.994 0.953 0.998 0.997 0.939
  0.988 0.985]
 [0.978 0.895 0.946 0.938 0.974 1.    0.943 0.99  0.983 0.963 0.969 0.842
  0.929 0.973]
 [0.983 0.992 1.    0.996 0.993 0.943 1.    0.978 0.916 0.995 0.99  0.971
  0.998 0.976]
 [0.996 0.944 0.979 0.977 0.994 0.99  0.978 1.    0.96  0.986 0.986 0.904
  0.967 0.993]
 [0.942 0.871 0.922 0.897 0.953 0.983 0.916 0.96  1.    0.949 0.961 0.811
  0.909 0.926]
 [0.985 0.98  0.997 0.987 0.998 0.963 0.995 0.986 0.949 1.    0.999 0.95
  0.993 0.975]
 [0.98  0.97  0.992 0.978 0.997 0.969 0.99  0.986 0.961 0.999 1.    0.935
  0.987 0.969]
 [0.926 0.993 0.968 0.